In [ ]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# first we fit the scaler on the training dataset
scaler.fit(train_data)

# then we call the transform method to scale both the training and testing data
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

# a sample output
print(train_data[0])

In [ ]:
test_data[0]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae',])
    return model

In [ ]:
num_epochs = 500
all_mae_histories = []

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4)
for train_index, val_index in kf.split(train_data):
    print('processing fold #', val_index)
    val_data, val_targets = train_data[val_index], train_targets[val_index]
    
    partial_train_data, partial_train_targets = train_data[train_index], train_targets[train_index]

    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data = (val_data, val_targets),
                        epochs = num_epochs, batch_size = 1, verbose = 0)

    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

In [ ]:
import numpy as np
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
#plt.plot(range(1, 101), average_mae_history[:100])
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=10)

In [ ]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
print(test_mse_score,test_mae_score)

In [ ]:
to_predict = test_data[[1, 25, 50, 75, 100]]
predictions = model.predict(to_predict)
print(predictions)

print(test_targets[[1, 25, 50, 75, 100]])